## Theano Introduction
1. Simple Model
    * Dense layer implementation
    * Test using some data set
    * Dropout layer
2. Convolution
    * Convolution 2D layer
    * Test using keras 2d data set

In [1]:
from keras.datasets import boston_housing
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

Using Theano backend.


In [2]:
#So theano is doing symbolic computation
#for my dense layer I want to define my input array 
#(lets say this is a 64 neuron dense layer)
import theano
from theano import shared
import theano.tensor as T
import theano.tensor.nnet as N
import math
import numpy as np

#taken from fast.ai lesson6 notebook
def init_wgts(rows, cols): 
    scale = math.sqrt(2/rows)
    return shared(np.random.normal(scale=scale, size=(rows, cols)).astype(np.float32))
def init_bias(rows): 
    return shared(np.zeros(rows, dtype=np.float32))

In [8]:
inp = T.vector() # input array of weights (batch size 1)
weights = T.matrix() # array of weights per neuron 
biases = T.vector() # array of biases per neuron
tar = T.scalar() #target 
lr = T.scalar() # learning rate

#shared variables
layer_weights= init_wgts(len(x_train[0]),64)
layer_biases = init_bias(64)

def step(x, weights,biases):
    return N.relu(x.dot(weights) + biases)

#for each neuron we want to do the activation(input * weights + bias)
results, updates = theano.scan(step, sequences=inp, 
                              non_sequences=[layer_weights,layer_biases])
summed = T.sum(results)
error = T.sqrt(T.sqr(summed-tar))

#How to calculate gradient?
gy = T.grad(error, [layer_weights, layer_biases])

In [9]:
from collections import *
#update function?
def upd_dict(weights, biases, grads, lr):
    return [(weights, weights + (-1 * lr * grads[0])),
                        (biases, biases + (-1 * lr * grads[1]))]
upd = upd_dict(layer_weights, layer_biases, gy, lr)

In [10]:
#goal try to write a function for this update
#so my problem was not using shared variables for state
#T.grad also needs to use shared variables
fn = theano.function([inp,weights,biases,lr], error, updates=upd)

UnusedInputError: theano.function was asked to create a function computing outputs given certain inputs, but the provided input variable at index 1 is not part of the computational graph needed to compute the outputs: <TensorType(float32, matrix)>.
To make this error into a warning, you can pass the parameter on_unused_input='warn' to theano.function. To disable it completely, use on_unused_input='ignore'.

In [9]:
len(x_train[0])

13